In [1]:
import logging
import os
import sqlite3
from dataclasses import dataclass, field
from typing import Optional, List
from datetime import datetime
from dotenv import load_dotenv
import asyncio
from livekit.agents.llm import function_tool



In [2]:
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

llm = ChatGroq(model = "gemma2-9b-it",api_key="")

Agent1 = create_react_agent(
    model=llm,
    tools=[multiply]
)

In [5]:
logger = logging.getLogger("nurse-assistant")
logger.setLevel(logging.INFO)

load_dotenv()

def init_db():
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()

    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS patients (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            phone TEXT UNIQUE NOT NULL,
            email TEXT UNIQUE NOT NULL,
            insurance_provider TEXT,
            insurance_number TEXT
        )
    """)

  
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS doctors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            specialty TEXT NOT NULL
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS appointments (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            doctor_id INTEGER,
            specialty TEXT NOT NULL,
            preferred_date TEXT NOT NULL,
            preferred_time TEXT NOT NULL,
            FOREIGN KEY (patient_id) REFERENCES patients(id),
            FOREIGN KEY (doctor_id) REFERENCES doctors(id)
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS insurance_claims (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            insurance_provider TEXT,
            insurance_number TEXT,
            claim_amount REAL,
            status TEXT,
            FOREIGN KEY (patient_id) REFERENCES patients(id)
        )
    """)

   
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS medicines (
            name TEXT PRIMARY KEY,
            description TEXT,
            side_effects TEXT
        )
    """)

    # Insert sample doctor data
    doctors_data = [
        ("Dr. Anil Sharma", "General Medicine"), ("Dr. Priya Gupta", "General Medicine"),
        ("Dr. Rajesh Kumar", "General Medicine"), ("Dr. Neha Patel", "General Medicine"),
        ("Dr. Sanjay Desai", "General Medicine"), ("Dr. Anjali Mehta", "General Medicine"),
        ("Dr. Vikram Singh", "General Medicine"), ("Dr. Pooja Shah", "General Medicine"),
        ("Dr. Rakesh Verma", "General Medicine"), ("Dr. Sunita Joshi", "General Medicine"),
        ("Dr. Amit Choudhary", "Orthopedics"), ("Dr. Shalini Kapoor", "Orthopedics"),
        ("Dr. Manoj Patil", "Orthopedics"), ("Dr. Kavita Rana", "Orthopedics"),
        ("Dr. Deepak Malhotra", "Orthopedics"), ("Dr. Meera Nair", "Orthopedics"),
        ("Dr. Rohan Kulkarni", "Orthopedics"), ("Dr. Swati Thakur", "Orthopedics"),
        ("Dr. Vinod Agarwal", "Orthopedics"), ("Dr. Lakshmi Iyer", "Orthopedics"),
        ("Dr. Sameer Khan", "Psychiatry"), ("Dr. Ritu Saxena", "Psychiatry"),
        ("Dr. Arjun Menon", "Psychiatry"), ("Dr. Nisha Varghese", "Psychiatry"),
        ("Dr. Siddharth Bose", "Psychiatry"), ("Dr. Ananya Das", "Psychiatry"),
        ("Dr. Karan Oberoi", "Psychiatry"), ("Dr. Preeti Malhotra", "Psychiatry"),
        ("Dr. Vivek Sharma", "Psychiatry"), ("Dr. Smriti Jain", "Psychiatry"),
        ("Dr. Rahul Mehra", "Cardiology"), ("Dr. Suman Gupta", "Cardiology"),
        ("Dr. Ashok Reddy", "Cardiology"), ("Dr. Divya Sharma", "Cardiology"),
        ("Dr. Kunal Desai", "Cardiology"), ("Dr. Rekha Pillai", "Cardiology"),
        ("Dr. Manish Thakur", "Cardiology"), ("Dr. Seema Kapoor", "Cardiology"),
        ("Dr. Ajay Bhatt", "Cardiology"), ("Dr. Lakshmi Nair", "Cardiology"),
        ("Dr. Vikrant Singh", "Neurology"), ("Dr. Anjali Rao", "Neurology"),
        ("Dr. Sanjay Gupta", "Neurology"), ("Dr. Priyanka Shah", "Neurology"),
        ("Dr. Rohit Kumar", "Neurology"), ("Dr. Neeta Patel", "Neurology"),
        ("Dr. Aravind Menon", "Neurology"), ("Dr. Shalini Desai", "Neurology"),
        ("Dr. Rajiv Malhotra", "Neurology"), ("Dr. Meena Iyer", "Neurology")
    ]
    cursor.executemany("INSERT OR IGNORE INTO doctors (name, specialty) VALUES (?, ?)", doctors_data)

    # Insert sample patient data
    patients_data = [
        ("Arav Saxena", "+919876543210", "arav.saxena@example.com", "Star Health", "SH123456"),
        ("Priya Sharma", "+918765432109", "priya.sharma@example.com", "HDFC Ergo", "HE789012"),
        ("Rahul Mehta", "+917654321098", "rahul.mehta@example.com", None, None)
    ]
    cursor.executemany(
        "INSERT OR IGNORE INTO patients (name, phone, email, insurance_provider, insurance_number) VALUES (?, ?, ?, ?, ?)",
        patients_data
    )

    # Insert sample appointment data
    cursor.execute("SELECT id FROM patients WHERE name = 'Arav Saxena'")
    arav_id = cursor.fetchone()[0]
    cursor.execute("SELECT id FROM doctors WHERE name = 'Dr. Anil Sharma' AND specialty = 'General Medicine'")
    doctor1_id = cursor.fetchone()[0]
    cursor.execute("SELECT id FROM doctors WHERE name = 'Dr. Amit Choudhary' AND specialty = 'Orthopedics'")
    doctor2_id = cursor.fetchone()[0]
    appointments_data = [
        (arav_id, doctor1_id, "General Medicine", "2025-06-15", "10:00"),
        (arav_id, doctor2_id, "Orthopedics", "2025-06-20", "14:30"),
        (2, doctor1_id, "General Medicine", "2025-06-16", "11:00"), 
        (3, doctor2_id, "Orthopedics", "2025-06-18", "09:30")  
    ]
    cursor.executemany(
        "INSERT INTO appointments (patient_id, doctor_id, specialty, preferred_date, preferred_time) VALUES (?, ?, ?, ?, ?)",
        appointments_data
    )

   
    insurance_claims_data = [
        (arav_id, "Star Health", "SH123456", 5000.0, "Pending"),
        (2, "HDFC Ergo", "HE789012", 7500.0, "Approved")
    ]
    cursor.executemany(
        "INSERT INTO insurance_claims (patient_id, insurance_provider, insurance_number, claim_amount, status) VALUES (?, ?, ?, ?, ?)",
        insurance_claims_data
    )

    medicines_data = [
        ("Paracetamol", "Pain reliever and fever reducer", "Nausea, rash, liver damage (rare)"),
        ("Ibuprofen", "Nonsteroidal anti-inflammatory drug", "Stomach pain, dizziness, headache"),
        ("Aspirin", "Pain reliever and blood thinner", "Stomach upset, bleeding risk"),
        ("Amoxicillin", "Antibiotic for bacterial infections", "Diarrhea, rash, allergic reactions")
    ]
    cursor.executemany(
        "INSERT OR IGNORE INTO medicines (name, description, side_effects) VALUES (?, ?, ?)",
        medicines_data
    )

    conn.commit()
    conn.close()

init_db()

In [6]:
@function_tool
async def identify_patient(self, name: str, phone: str, email: str, insurance_provider: Optional[str] = None, insurance_number: Optional[str] = None) -> str:
    """Identify a patient by their name, phone, email, and optional insurance details."""
    userdata: UserData = self.session.userdata
    if not re.match(r'^(?:\+91[-\s]?)?[6789]\d{9}$', phone):
        return "Please provide a valid phone number (e.g., +919876543210)."
    if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
        return "Please provide a valid email address (e.g., example@domain.com)."

    try:
        conn = sqlite3.connect("hospital.db")
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO patients (name, phone, email, insurance_provider, insurance_number) VALUES (?, ?, ?, ?, ?)",
            (name, phone, email, insurance_provider, insurance_number)
        )
        conn.commit()
        patient_id = cursor.lastrowid
        conn.close()

        userdata.name = name
        userdata.phone = phone
        userdata.email = email
        userdata.patient_id = patient_id
        userdata.insurance_provider = insurance_provider
        userdata.insurance_number = insurance_number

        return f"Thank you, {name}. I've registered your details."
    except sqlite3.IntegrityError:
        return "This phone number or email is already registered. Please provide unique details."

@function_tool
async def assess_injury(self, symptoms: str) -> str:
    """Assess physical injury symptoms and suggest a specialty."""
    # Enhanced symptom-to-specialty mapping
    symptom_map = {
        "pain in arm": "Orthopedics",
        "leg pain": "Orthopedics",
        "back pain": "Orthopedics",
        "sports injury": "Orthopedics",
        "fracture": "Orthopedics",
        "chest pain": "Cardiology",
        "heart": "Cardiology",
        "headache": "Neurology",
        "seizure": "Neurology",
        "fever": "General Medicine",
        "cough": "General Medicine",
        "fall": "Orthopedics",
        "accident": "Orthopedics"
    }
    specialty = None
    for symptom, spec in symptom_map.items():
        if symptom.lower() in symptoms.lower():
            specialty = spec
            break
    if not specialty:
        specialty = "General Medicine"

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM doctors WHERE specialty = ? LIMIT 10", (specialty,))
    doctors = [row[0] for row in cursor.fetchall()]
    conn.close()

    doctor_list = ", ".join(doctors) if doctors else "No doctors available."
    return (
        f"Based on your symptoms ('{symptoms}'), I recommend seeing a {specialty} specialist. "
        f"Available doctors: {doctor_list}. Would you like to book an appointment?"
    )


In [7]:
@function_tool
async def assess_mental_health(self, symptoms: str) -> str:
    """Assess mental health symptoms and suggest a specialty."""
    if any(keyword in symptoms.lower() for keyword in ["anxiety", "depression", "stress", "mood", "sleep"]):
        specialty = "Psychiatry"
    else:
        specialty = "General Medicine"

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM doctors WHERE specialty = ? LIMIT 10", (specialty,))
    doctors = [row[0] for row in cursor.fetchall()]
    conn.close()

    doctor_list = ", ".join(doctors) if doctors else "No doctors available."
    return (
        f"Based on your symptoms ('{symptoms}'), I recommend seeing a {specialty} specialist. "
        f"Available doctors: {doctor_list}. Would you like to book an appointment?"
    )
@function_tool
async def book_appointment(self, specialty: str, preferred_date: str, preferred_time: str, insurance_provider: Optional[str] = None, insurance_number: Optional[str] = None) -> str:
    """Book an appointment for a patient with a specific specialty."""
    userdata: UserData = self.session.userdata
    if not userdata.is_identified():
        return "Please identify yourself first using name, phone, and email."

    try:
        datetime.strptime(preferred_date, "%Y-%m-%d")
        datetime.strptime(preferred_time, "%H:%M")
    except ValueError:
        return "Please provide date in YYYY-MM-DD format and time in HH:MM format."

    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM doctors WHERE specialty = ? LIMIT 1", (specialty,))
    doctor = cursor.fetchone()
    if not doctor:
        conn.close()
        return f"Specialty '{specialty}' is not available. Please choose another specialty."

    doctor_id, doctor_name = doctor

    # Update insurance details if provided
    if insurance_provider and insurance_number:
        cursor.execute(
            "UPDATE patients SET insurance_provider = ?, insurance_number = ? WHERE id = ?",
            (insurance_provider, insurance_number, userdata.patient_id)
        )
        conn.commit()
        userdata.insurance_provider = insurance_provider
        userdata.insurance_number = insurance_number

    cursor.execute(
        "INSERT INTO appointments (patient_id, doctor_id, specialty, preferred_date, preferred_time) VALUES (?, ?, ?, ?, ?)",
        (userdata.patient_id, doctor_id, specialty, preferred_date, preferred_time)
    )
    conn.commit()
    booking_id = cursor.lastrowid
    conn.close()

    email_sent = await self._send_confirmation_email(userdata.email, booking_id, specialty, preferred_date, preferred_time)
    userdata.current_booking = None

    if email_sent:
        return (
            f"Great! Your appointment (#{booking_id}) has been confirmed for {preferred_date} at {preferred_time} "
            f"with {doctor_name} ({specialty}). You'll receive a confirmation email."
        )
    return (
        f"Great! Your appointment (#{booking_id}) has been confirmed for {preferred_date} at {preferred_time} "
        f"with {doctor_name} ({specialty}). However, there was an issue sending the confirmation email. "
        f"Please check your email later or contact us if you don’t receive it."
    )



In [8]:
@function_tool
async def view_appointments(self, phone: str) -> str:
    """View upcoming appointments for a patient by phone number."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT a.id, d.name, a.specialty, a.preferred_date, a.preferred_time 
        FROM appointments a 
        JOIN patients p ON a.patient_id = p.id 
        JOIN doctors d ON a.doctor_id = d.id 
        WHERE p.phone = ?
        """,
        (phone,)
    )
    bookings = cursor.fetchall()
    conn.close()

    if not bookings:
        return "You have no upcoming appointments."
    appointment_list = "\n".join([
        f"- Appointment #{b[0]} with {b[1]} ({b[2]}) on {b[3]} at {b[4]}" for b in bookings
    ])
    return f"You have the following upcoming appointments:\n{appointment_list}"

In [9]:
!pip install "livekit-agents[google]~=1.0"

  Using cached livekit_plugins_google-1.1.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/216.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/216.1 kB 1.7 MB/s eta 0:00:01
   ------------ -------------------------- 71.7/216.1 kB 787.7 kB/s eta 0:00:01
   ------------------- ------------------ 112.6/216.1 kB 930.9 kB/s eta 0:00:01
   ------------------- ------------------ 112.6/216.1 kB 930.9 kB/s eta 0:00:01
   ------------------- ------------------ 112.6/216.1 kB 930.9 kB/s eta 0:00:01
   --------------------------- ---------- 153.6/216.1 kB 573.4 kB/s eta 0:00:01
   --------------------------- 

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\aravs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
@function_tool
async def update_appointment(self, booking_id: int, new_date: Optional[str] = None, new_time: Optional[str] = None, new_specialty: Optional[str] = None) -> str:
    """Update an existing appointment by booking ID."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute(
        "SELECT patient_id, doctor_id, specialty, preferred_date, preferred_time FROM appointments WHERE id = ?",
        (booking_id,)
    )
    booking = cursor.fetchone()
    if not booking:
        conn.close()
        return "Appointment not found. Please check the booking ID."

    patient_id, current_doctor_id, current_specialty, current_date, current_time = booking
    new_specialty = new_specialty or current_specialty
    new_date = new_date or current_date
    new_time = new_time or current_time

    cursor.execute("SELECT id FROM doctors WHERE specialty = ? LIMIT 1", (new_specialty,))
    doctor = cursor.fetchone()
    if not doctor:
        conn.close()
        return f"Specialty '{new_specialty}' is not available. Please choose another specialty."

    new_doctor_id = doctor[0]

    try:
        datetime.strptime(new_date, "%Y-%m-%d")
        datetime.strptime(new_time, "%H:%M")
    except ValueError:
        conn.close()
        return "Please provide date in YYYY-MM-DD format and time in HH:MM format."

    cursor.execute(
        "UPDATE appointments SET doctor_id = ?, specialty = ?, preferred_date = ?, preferred_time = ? WHERE id = ?",
        (new_doctor_id, new_specialty, new_date, new_time, booking_id)
    )
    conn.commit()
    conn.close()
    return "Appointment updated successfully."

@function_tool
async def cancel_appointment(self, booking_id: int) -> str:
    """Cancel an appointment by booking ID."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM appointments WHERE id = ?", (booking_id,))
    if cursor.rowcount == 0:
        conn.close()
        return "Appointment not found. Please check the booking ID."
    conn.commit()
    conn.close()
    return "Appointment canceled successfully."

@function_tool
async def check_insurance(self, phone: str) -> str:
    """Check if a patient has health insurance by phone number."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, insurance_provider, insurance_number FROM patients WHERE phone = ?", (phone,))
    patient = cursor.fetchone()
    conn.close()

    if not patient:
        return "No patient found with this phone number."
    patient_id, insurance_provider, insurance_number = patient
    if insurance_provider and insurance_number:
        return f"Insurance found: Provider={insurance_provider}, Policy Number={insurance_number}."
    return "No insurance details found for this patient."

@function_tool
async def submit_insurance_claim(self, phone: str, claim_amount: float) -> str:
    """Submit an insurance claim for a patient."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, insurance_provider, insurance_number FROM patients WHERE phone = ?", (phone,))
    patient = cursor.fetchone()
    if not patient:
        conn.close()
        return "No patient found with this phone number."

    patient_id, insurance_provider, insurance_number = patient
    if not insurance_provider or not insurance_number:
        conn.close()
        return "No insurance details found. Please provide insurance information first."

    cursor.execute(
        "INSERT INTO insurance_claims (patient_id, insurance_provider, insurance_number, claim_amount, status) "
        "VALUES (?, ?, ?, ?, ?)",
        (patient_id, insurance_provider, insurance_number, claim_amount, "Pending")
    )
    conn.commit()
    claim_id = cursor.lastrowid
    conn.close()
    return f"Insurance claim #{claim_id} submitted for {claim_amount} INR. Status: Pending."
@function_tool
async def get_medicine_info(self, name: str) -> str:
    """Get information about a specific medicine."""
    conn = sqlite3.connect("hospital.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name, description, side_effects FROM medicines WHERE name = ?", (name,))
    medicine = cursor.fetchone()
    conn.close()

    if not medicine:
        return f"No information found for medicine: {name}"
    return (
        f"Medicine: {medicine[0]}\n"
        f"Description: {medicine[1]}\n"
        f"Side Effects: {medicine[2]}"
    )




In [ ]:

async def _send_confirmation_email(self, patient_email: str, booking_id: int, specialty: str, preferred_date: str, preferred_time: str) -> bool:
    """Send a confirmation email for the appointment."""
    try:
        email_sender = os.getenv("EMAIL_SENDER")
        email_password = os.getenv("EMAIL_PASSWORD")
        if not email_sender or not email_password:
            logger.error("Email credentials are missing in the environment variables.")
            return False

        msg = EmailMessage()
        msg['Subject'] = "Appointment Confirmation"
        msg['From'] = email_sender
        msg['To'] = patient_email
        msg.set_content(
            f"Hello {self.session.userdata.name}, your appointment (#{booking_id}) is scheduled for "
            f"{preferred_date} at {preferred_time} with a {specialty} specialist."
        )

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(email_sender, email_password)
            server.send_message(msg)

        logger.info(f"Confirmation email sent to {patient_email}")
        return True
    except Exception as e:
        logger.error(f"Failed to send email to {patient_email}: {e}")
        return False
